In [ ]:
%pip install torchsampler

In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

train_dir = './train_images'
test_dir = './test_images'

# Define the transformations for augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(), 
        transforms.RandomResizedCrop(36),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0,),std=(1,))
        # that normalize is useless, data will become (data-0)/1
    ]),
    'test':transforms.Compose([
        transforms.Grayscale(), 
        transforms.ToTensor(), 
        transforms.Normalize(mean=(0,),std=(1,))])        
}
train_data_augmented = torchvision.datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_data = torchvision.datasets.ImageFolder(test_dir, transform=data_transforms['test'])


In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
from torchsampler import ImbalancedDatasetSampler

batch_size = 32

train_loader = torch.utils.data.DataLoader(train_data_augmented, batch_size=batch_size, sampler=ImbalancedDatasetSampler(train_data_augmented), num_workers=2)
valid_loader = torch.utils.data.DataLoader(train_data_augmented, batch_size=batch_size, sampler=ImbalancedDatasetSampler(train_data_augmented), num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=2)
classes = ('noface','face')

In [8]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16) # Add a new fully connected layer
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = Net().to(device)

In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4) # Use Adam optimization

n_epochs = 64
epsilon = 0.1
early_stop = False

for epoch in range(1, n_epochs+1):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            if (running_loss / 2000) < epsilon:
                early_stop = True
                break
            running_loss = 0.0
    
    if early_stop:
        break

print('Finished Training')

[2,  2000] loss: 0.397
[3,  2000] loss: 0.227
[4,  2000] loss: 0.188
[5,  2000] loss: 0.167
[6,  2000] loss: 0.153
[7,  2000] loss: 0.142
[8,  2000] loss: 0.133
[9,  2000] loss: 0.129
[10,  2000] loss: 0.118
[11,  2000] loss: 0.115
[12,  2000] loss: 0.113
[13,  2000] loss: 0.109
[14,  2000] loss: 0.107
[15,  2000] loss: 0.102
[16,  2000] loss: 0.104
[17,  2000] loss: 0.101
[18,  2000] loss: 0.103
[19,  2000] loss: 0.101
[20,  2000] loss: 0.099
Finished Training


In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [18]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the {} test images: {:.2f}'.format(
    len(test_loader) * batch_size, 100 * correct / total))

Accuracy of the network on the 7648 test images: 98.3875
